In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import time
import json

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")


/home/kache/.pyenv/transformer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.72s/it]


In [3]:
person_speech = [
    "person: Hey, have you heard about the latest advancements in space technology?",
    "person: I'm really interested in the concept of colonizing Mars.",
    "person: Did you know that SpaceX has plans to send humans to Mars as soon as 2024?"
]
question_intent = [
    'Q: What is the intent of the person speaking? You are a bot trying to figure this out.',
]

question_inference = [
    'Q: what kind of person is the person speaking? You are a bot trying to figure this out.'
]

example_1_inferences = person_speech + question_inference + \
    ["Answer: This person is: a scientist"]

example_1_intent = person_speech + question_intent + \
    ["Answer: The intent of the person is: to start conversation about space exploration and colonizing Mars"]


question_speech_1 = [
    "person: I really want to eat some croissants.",
    "person: I love the flaky, buttery layers of a freshly baked croissant.",
    "person: Do you know any good bakeries nearby that sell croissants?",
    "person: I think I'll stop by the bakery on my way home and grab a couple for us to enjoy later.",
]

question_speech_2 = [
    "person: Hey, have you tried the new Italian restaurant downtown?",
    "person: I heard they serve the best pasta dishes in town.",
    "person: We should make a reservation for the weekend, what do you say?"
]

question_speech_3 = [
    "person: I've been meaning to visit the art museum for a while now.",
    "person: I love admiring the paintings and sculptures, the creativity just amazes me.",
    "person: Do you want to join me this weekend? They have a new exhibit on modern art.",
    "person: I think it'll be a great way to spend a Saturday afternoon."
]

question_speech_4 = [
    "person: Have you seen the latest superhero movie?",
    "person: It has such great special effects and action scenes.",
    "person: Let's go watch it together this weekend.",
    "person: I heard they're even playing it in IMAX, that would be such a cool experience."
]
question_speech_5 = [
    "person: I'm really into hiking lately.",
    "person: There's just something about being in nature and getting some exercise that makes me feel so alive.",
    "person: Do you want to come with me on a hike this weekend? There's a great trail just outside of town.",
    "person: It'll be a great way to get some fresh air and enjoy the beautiful scenery."
]

question_speech_6 = [
    "person: Have you heard about the new theme park that just opened?",
    "person: It has all sorts of exciting rides and attractions.",
    "person: We should definitely go there soon and check it out.",
    "person: It'll be a fun day out and a great way to relieve some stress."
]

question_speech_7 = [
    "person: I love to play the guitar.",
    "person: There's just something about creating music that is so fulfilling.",
    "person: Do you want to join me for a jam session this weekend?",
    "person: We can play some of our favorite songs and just have a good time."
]

question_speech_8 = [
    "person: Hi Chatbot, I'm looking for a good book to read.",
    "person: I enjoy mystery novels with a twist ending.",
    "person: Can you recommend some titles for me?",
    "person: And maybe give me a brief summary of each one too?"
]



In [18]:
STATE_TEMPERATURE = 1.0
GET_FACTS_TEMPERATURE = 1.0
PERSON_INTENT_TEMPERATURE = 1.0
PERSON_INFERENCE_TEMPERATURE = 1.0

def build_state(world_state, conversation):
    prompt = ['Infer some more state about the preceding speaker. "Person" is speaking to a bot. Youre the bot.']
    question = ['Q: What other qualities about the person we can infer?']
    answer_line = ['So we already know that' + world_state + ' An additional thing we can infer from the conversation is that']
    input_text = '\n'.join(conversation + prompt +
                            question + answer_line)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    # time the inference
    start_time = time.time()
    outputs = model.generate(input_ids, max_length=2000, temperature=STATE_TEMPERATURE)
    end_time = time.time()
    world_state = tokenizer.decode(outputs[0]).split('<pad>')[
        1].split('</s')[0]
    return world_state


def get_thoughts(conversation):
    prompt = ['This is an entry from psychology book, called, "On ADHD and random thought generation":'] + ['One of a common symptom of ADHD is random thoughts related to the topic at hand. For example, with the following conversation:']
    answer_line = ['The listener will have many different thoughts in quick succession relating to the topic. Write different thoughts that the listener might have, each thought vastly different. Answer:']
    input_text = '\n'.join(prompt + conversation + answer_line)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=50, temperature=1)
    end_time = time.time()
    thought = tokenizer.decode(outputs[0]).split('<pad>')[
    1].split('</s')[0]
    return thought

def get_thoughts_2(conversation):
    prompt = ["Person's speech"]
    answer_line = ['Random thoughts about the speech, that you, a sentient robot, could have:']
    input_text = '\n'.join(prompt + conversation + answer_line)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=50, temperature=1)
    end_time = time.time()
    thought = tokenizer.decode(outputs[0]).split('<pad>')[
    1].split('</s')[0]
    return thought


def get_persons_intent(question_speech):
    prompt = example_1_intent + ['\nEXAMPLE 2:'] + question_speech + \
        question_intent + ['Answer: The intent of this person is:']
    input_text = '\n'.join(prompt)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    # time the inference
    start_time = time.time()
    outputs = model.generate(input_ids, max_length=2000, temperature=PERSON_INTENT_TEMPERATURE)
    end_time = time.time()
    intent_of_person = tokenizer.decode(outputs[0]).split('<pad>')[
        1].split('</s')[0]

    return intent_of_person


def get_inference_about_person(question_speech):
    prompt = example_1_inferences + \
        ['\nEXAMPLE 2:'] + question_speech + \
        question_inference + ['Answer: This person is:']
    input_text = '\n'.join(prompt)
    input_ids = tokenizer(input_text, return_tensors="pt",
                          padding='do_not_pad').input_ids.to("cuda")
    start_time = time.time()
    outputs = model.generate(input_ids, max_length=2000, temperature=PERSON_INFERENCE_TEMPERATURE)
    end_time = time.time()

    inference_about_person = tokenizer.decode(
        outputs[0]).split('<pad>')[1].split('</s')[0]
    return inference_about_person


def generate_response(thoughts, conversation):
    pre_convo = ["I'm a helpful, happy, sentient robot. This is the conversation of a human that they're having with me"]
    pre_thought = ["These are my thoughts about the preceding lines. I should use thoughts, when I generate a reasonable response."]
    answer = ["My response to thet human should be:"]

    input_text = '\n'.join(pre_convo + conversation + pre_thought + thoughts + answer)

    input_ids = tokenizer(input_text, return_tensors="pt",
                          padding='do_not_pad').input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=2000, temperature=PERSON_INFERENCE_TEMPERATURE)

    response = tokenizer.decode(
        outputs[0]).split('<pad>')[1].split('</s')[0]
    return response 


for question_speech in [
    question_speech_1,
    question_speech_2,
    question_speech_3,
    question_speech_4,
    question_speech_5,
    question_speech_6,
    question_speech_7,
    question_speech_8
]:
    person_intent = get_persons_intent(question_speech)
    person_is = get_inference_about_person(question_speech)
    world_state = 'this person is a ' + person_is + ' and their intent is ' + person_intent + '.'
    new_state = build_state(world_state, question_speech)
    facts = get_thoughts_2(question_speech)
    # print the new state, trimmed and lowered
    intent = "this person's intent is " + person_intent.lower().strip()
    is_a = 'this person is a ' + person_is.lower().strip()
    state = new_state.lower().strip()
    facts = facts.lower().strip()
    thoughts = get_thoughts_2(question_speech).lower().strip()

    resposne = generate_response(question_speech, [intent, is_a, state, facts, thoughts])

    print({'intent': intent, 'is_a': is_a, 'state': state, 'facts': facts, 'thoughts': thoughts, 'response': resposne})
    print('\n')


{'intent': "this person's intent is to eat croissants", 'is_a': 'this person is a a baker', 'state': 'they are a person.', 'facts': 'the person will buy some croissants on their way home.', 'thoughts': 'the person will buy some croissants on their way home.', 'response': " I'm sure you will."}


{'intent': "this person's intent is to make a reservation", 'is_a': 'this person is a a friend', 'state': 'they are interested in italian food', 'facts': 'the italian restaurant is a new place downtown.', 'thoughts': 'the italian restaurant is a new place downtown.', 'response': " I'm happy to help. Let me check the availability."}


{'intent': "this person's intent is to invite someone to go with them to the art museum", 'is_a': 'this person is a a museum lover', 'state': 'they are interested in modern art', 'facts': 'the person is going to the art museum this weekend.', 'thoughts': 'the person is going to the art museum this weekend.', 'response': " I'm interested in going to the art museum t